In [76]:
import h2o
from h2o.automl import H2OAutoML
import random, os, sys
from datetime import datetime
import pandas as pd
import logging
import csv
import optparse
import time
import json
from distutils.util import strtobool
import psutil

In [77]:
data_path=None
all_variables=None
test_path=None
# target='search_term'
target=None
nthreads=1 
min_mem_size=6 
run_time=100
classification=False
scale=False
max_models=9    
model_path=None
balance_y=False 
balance_threshold=0.2
name=None 
server_path=None  
analysis=0 

In [78]:
pct_memory=0.5
virtual_memory=psutil.virtual_memory()
min_mem_size=int(round(int(pct_memory*virtual_memory.available)/1073741824,0))
print(min_mem_size)

4


In [79]:
# Functions

def alphabet(n):
  alpha='0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'    
  str=''
  r=len(alpha)-1   
  while len(str)<n:
    i=random.randint(0,r)
    str+=alpha[i]   
  return str
  
  
def set_meta_data(analysis,run_id,server,data,test,model_path,target,run_time,classification,scale,model,balance,balance_threshold,name,path,nthreads,min_mem_size):
  m_data={}
  m_data['start_time'] = time.time()
  m_data['target']=target
  m_data['server_path']=server
  m_data['data_path']=data 
  m_data['test_path']=test
  m_data['max_models']=model
  m_data['run_time']=run_time
  m_data['run_id'] =run_id
  m_data['scale']=scale
  m_data['classification']=classification
  m_data['scale']=False
  m_data['model_path']=model_path
  m_data['balance']=balance
  m_data['balance_threshold']=balance_threshold
  m_data['project'] =name
  m_data['end_time'] = time.time()
  m_data['execution_time'] = 0.0
  m_data['run_path'] =path
  m_data['nthreads'] = nthreads
  m_data['min_mem_size'] = min_mem_size
  m_data['analysis'] = analysis
  return m_data


def dict_to_json(dct,n):
  j = json.dumps(dct, indent=4)
  f = open(n, 'w')
  print(j, file=f)
  f.close()
  
  
def stackedensemble(mod):
    coef_norm=None
    try:
      metalearner = h2o.get_model(mod.metalearner()['name'])
      coef_norm=metalearner.coef_norm()
    except:
      pass        
    return coef_norm

def stackedensemble_df(df):
    bm_algo={ 'GBM': None,'GLM': None,'DRF': None,'XRT': None,'Dee': None}
    for index, row in df.iterrows():
      if len(row['model_id'])>3:
        key=row['model_id'][0:3]
        if key in bm_algo:
          if bm_algo[key] is None:
                bm_algo[key]=row['model_id']
    bm=list(bm_algo.values()) 
    bm=list(filter(None.__ne__, bm))             
    return bm

def se_stats(modl):
    d={}
    d['algo']=modl.algo
    d['model_id']=modl.model_id   
    d['auc']=modl.auc()   
    d['roc']=modl.roc()
    d['mse']=modl.mse()   
    d['null_degrees_of_freedom']=modl.null_degrees_of_freedom()
    d['null_deviance']=modl.null_deviance()
    d['residual_degrees_of_freedom']=modl.residual_degrees_of_freedom()   
    d['residual_deviance']=modl.residual_deviance()
    d['rmse']=modl.rmse()
    return d

def get_model_by_algo(algo,models_dict):
    mod=None
    mod_id=None    
    for m in list(models_dict.keys()):
        if m[0:3]==algo:
            mod_id=m
            mod=h2o.get_model(m)      
    return mod,mod_id     
    
    
def gbm_stats(modl):
    d={}
    d['algo']=modl.algo
    d['model_id']=modl.model_id   
    d['varimp']=modl.varimp()  
    return d
    
    
def dl_stats(modl):
    d={}
    d['algo']=modl.algo
    d['model_id']=modl.model_id   
    d['varimp']=modl.varimp()  
    return d
    
    
def drf_stats(modl):
    d={}
    d['algo']=modl.algo
    d['model_id']=modl.model_id   
    d['varimp']=modl.varimp()  
    d['roc']=modl.roc()      
    return d
    
def xrt_stats(modl):
    d={}
    d['algo']=modl.algo
    d['model_id']=modl.model_id   
    d['varimp']=modl.varimp()  
    d['roc']=modl.roc()      
    return d
    
    
def glm_stats(modl):
    d={}
    d['algo']=modl.algo
    d['model_id']=modl.model_id   
    d['coef']=modl.coef()  
    d['coef_norm']=modl.coef_norm()      
    return d
    
def model_performance_stats(perf):
    d={}
    try:    
      d['mse']=perf.mse()
    except:
      pass      
    try:    
      d['rmse']=perf.rmse() 
    except:
      pass      
    try:    
      d['null_degrees_of_freedom']=perf.null_degrees_of_freedom()
    except:
      pass      
    try:    
      d['residual_degrees_of_freedom']=perf.residual_degrees_of_freedom()
    except:
      pass      
    try:    
      d['residual_deviance']=perf.residual_deviance() 
    except:
      pass      
    try:    
      d['null_deviance']=perf.null_deviance() 
    except:
      pass      
    try:    
      d['aic']=perf.aic() 
    except:
      pass      
    try:
      d['logloss']=perf.logloss() 
    except:
      pass    
    try:
      d['auc']=perf.auc()
    except:
      pass  
    try:
      d['gini']=perf.gini()
    except:
      pass    
    return d
    
def impute_missing_values(df, x, scal=False):
    # determine column types
    ints, reals, enums = [], [], []
    for key, val in df.types.items():
        if key in x:
            if val == 'enum':
                enums.append(key)
            elif val == 'int':
                ints.append(key)            
            else: 
                reals.append(key)    
    _ = df[reals].impute(method='mean')
    _ = df[ints].impute(method='median')
    if scal:
        df[reals] = df[reals].scale()
        df[ints] = df[ints].scale()    
    return


def get_independent_variables(df, targ):
    C = [name for name in df.columns if name != targ]
    # determine column types
    ints, reals, enums = [], [], []
    for key, val in df.types.items():
        if key in C:
            if val == 'enum':
                enums.append(key)
            elif val == 'int':
                ints.append(key)            
            else: 
                reals.append(key)    
    x=ints+enums+reals
    return x
    
def get_all_variables_csv(i):
    ivd={}
    try:
      iv = pd.read_csv(i,header=None)
    except:
      sys.exit(1)    
    col=iv.values.tolist()[0]
    dt=iv.values.tolist()[1]
    i=0
    for c in col:
      ivd[c.strip()]=dt[i].strip()
      i+=1        
    return ivd
    
    

def check_all_variables(df,dct,y=None):     
    targ=list(dct.keys())     
    for key, val in df.types.items():
        if key in targ:
          if dct[key] not in ['real','int','enum']:                      
            targ.remove(key)  
    for key, val in df.types.items():
        if key in targ:            
          if dct[key] != val:
            print('convert ',key,' ',dct[key],' ',val)
            if dct[key]=='enum':
                try:
                  df[key] = df[key].asfactor() 
                except:
                  targ.remove(key)                 
            if dct[key]=='int': 
                try:                
                  df[key] = df[key].asnumeric() 
                except:
                  targ.remove(key)                  
            if dct[key]=='real':
                try:                
                  df[key] = df[key].asnumeric()  
                except:
                  targ.remove(key)                  
    if y is None:
      y=df.columns[-1] 
    if y in targ:
      targ.remove(y)
    else:
      y=targ.pop()            
    return targ    
    
def predictions(mod,data,run_id):
    test = h2o.import_file(data)
    mod_perf=mod_best.model_performance(test)
              
    stats_test={}
    stats_test=model_performance_stats(mod_perf)

    n=run_id+'_test_stats.json'
    dict_to_json(stats_test,n) 

    try:    
      cf=mod_perf.confusion_matrix(metrics=["f1","f2","f0point5","accuracy","precision","recall","specificity","absolute_mcc","min_per_class_accuracy","mean_per_class_accuracy"])
      cf_df=cf[0].table.as_data_frame()
      cf_df.to_csv(run_id+'_test_confusion_matrix.csv')
    except:
      pass

    predictions = mod_best.predict(test)
    predictions_df=test.cbind(predictions).as_data_frame() 
    predictions_df.to_csv(run_id+'_predictions.csv')
    return

def predictions_test(mod,test,run_id):
    mod_perf=mod_best.model_performance(test)          
    stats_test={}
    stats_test=model_performance_stats(mod_perf)
    n=run_id+'_test_stats.json'
    dict_to_json(stats_test,n) 
    try:
      cf=mod_perf.confusion_matrix()
#      cf=mod_perf.confusion_matrix(metrics=["f1","f2","f0point5","accuracy","precision","recall","specificity","absolute_mcc","min_per_class_accuracy","mean_per_class_accuracy"])
      cf_df=cf.table.as_data_frame()
      cf_df.to_csv(run_id+'_test_confusion_matrix.csv')
    except:
      pass
    predictions = mod_best.predict(test)    
    predictions_df=test.cbind(predictions).as_data_frame() 
    predictions_df.to_csv(run_id+'_predictions.csv')
    return predictions

def check_X(x,df):
    for name in x:
        if name not in df.columns:
          x.remove(name)  
    return x    
    
    
def get_stacked_ensemble(lst):
    se=None
    for model in model_set:
      if 'BestOfFamily' in model:
        se=model
    if se is None:     
      for model in model_set:
        if 'AllModels'in model:
          se=model           
    return se       
    
def get_variables_types(df):
    d={}
    for key, val in df.types.items():
        d[key]=val           
    return d    
    
#  End Functions

In [80]:
all_variables=None

In [82]:
data_path='C:/Users/prabh/Desktop/ML/Project ML/Dataset/Titanic_train.csv'

In [83]:
run_id=alphabet(9)
if server_path==None:
  server_path=os.path.abspath(os.curdir)
os.chdir(server_path) 
run_dir = os.path.join(server_path,run_id)
os.mkdir(run_dir)
os.chdir(run_dir)    

# run_id to std out
print (run_id) 

WXf3Ticdf


In [84]:
logfile=run_id+'_autoh2o_log.zip'
logs_path=os.path.join(run_dir,'logs')
print(logs_path,' ',logfile)

C:\Users\prabh\WXf3Ticdf\logs   WXf3Ticdf_autoh2o_log.zip


In [85]:
# 65535 Highest port no
port_no=random.randint(5555,55555)
h2o.init(strict_version_check=False,min_mem_size_GB=min_mem_size,port=port_no)

Checking whether there is an H2O instance running at http://localhost:12968..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM (build 25.191-b12, mixed mode)
  Starting server from C:\Users\prabh\Anaconda3\lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\prabh\AppData\Local\Temp\tmpjs_df56f
  JVM stdout: C:\Users\prabh\AppData\Local\Temp\tmpjs_df56f\h2o_prabh_started_from_python.out
  JVM stderr: C:\Users\prabh\AppData\Local\Temp\tmpjs_df56f\h2o_prabh_started_from_python.err
  Server is running at http://127.0.0.1:12968
Connecting to H2O server at http://127.0.0.1:12968... successful.


H2O cluster uptime:,01 secs
H2O cluster timezone:,America/New_York
H2O data parsing timezone:,UTC
H2O cluster version:,3.22.0.1
H2O cluster version age:,1 month and 17 days
H2O cluster name:,H2O_from_python_prabh_1tisdx
H2O cluster total nodes:,1
H2O cluster free memory:,3.833 Gb
H2O cluster total cores:,12
H2O cluster allowed cores:,12
H2O cluster status:,"accepting new members, healthy"


In [86]:
# meta data
meta_data = set_meta_data(analysis, run_id,server_path,data_path,test_path,model_path,target,run_time,classification,scale,max_models,balance_y,balance_threshold,name,run_dir,nthreads,min_mem_size)
print(meta_data)  

{'start_time': 1544825621.785323, 'target': None, 'server_path': 'C:\\Users\\prabh', 'data_path': 'C:/Users/prabh/Desktop/ML/Project ML/Dataset/Titanic_train.csv', 'test_path': None, 'max_models': 9, 'run_time': 100, 'run_id': 'WXf3Ticdf', 'scale': False, 'classification': False, 'model_path': None, 'balance': False, 'balance_threshold': 0.2, 'project': None, 'end_time': 1544825621.785323, 'execution_time': 0.0, 'run_path': 'C:\\Users\\prabh\\WXf3Ticdf', 'nthreads': 1, 'min_mem_size': 4, 'analysis': 0}


In [87]:
print(data_path)

C:/Users/prabh/Desktop/ML/Project ML/Dataset/Titanic_train.csv


In [88]:
df = h2o.import_file(data_path)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [89]:
df.head()

PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,nan,7.25,,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",female,38,1,0,nan,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,nan,7.925,,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,53.1,C123,S
5,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,8.05,,S
6,0,3,"Moran, Mr. James",male,nan,0,0,330877,8.4583,,Q
7,0,1,"McCarthy, Mr. Timothy J",male,54,0,0,17463,51.8625,E46,S
8,0,3,"Palsson, Master. Gosta Leonard",male,2,3,1,349909,21.075,,S
9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27,0,2,347742,11.1333,,S
10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14,1,0,237736,30.0708,,C


In [90]:
df.describe()

Rows:891
Cols:12




,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
type,int,int,int,string,enum,real,int,int,int,real,enum,enum
mins,1.0,0.0,1.0,NaN,,0.42,0.0,0.0,693.0,0.0,,
mean,446.0,0.3838383838383838,2.3086419753086447,NaN,,29.69911764705884,0.5230078563411893,0.3815937149270483,260318.5491679275,32.20420796857465,,
maxs,891.0,1.0,3.0,NaN,,80.0,8.0,6.0,3101298.0,512.3292,,
sigma,257.3538420152301,0.4865924542648575,0.8360712409770491,NaN,,14.526497332334035,1.1027434322934315,0.8060572211299488,471609.26868834975,49.69342859718089,,
zeros,0,549,0,0,,0,608,678,0,15,,
missing,0,0,0,0,0,177,0,0,230,0,687,2
0,1.0,0.0,3.0,"Braund, Mr. Owen Harris",male,22.0,1.0,0.0,nan,7.25,,S
1,2.0,1.0,1.0,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",female,38.0,1.0,0.0,nan,71.2833,C85,C
2,3.0,1.0,3.0,"Heikkinen, Miss. Laina",female,26.0,0.0,0.0,nan,7.925,,S


In [91]:
'''
import sys
sys.stdout = open('describe.txt', 'w')
print ('test')
'''

"\nimport sys\nsys.stdout = open('describe.txt', 'w')\nprint ('test')\n"

In [93]:
# dependent variable
# assign target and inputs for classification or regression
if target==None:
  target=df.columns[1]   
y = target

In [94]:
print(y)

Age


In [95]:
print(all_variables)

None


In [96]:
if all_variables is not None:
  ivd=get_all_variables_csv(all_variables)
  print(ivd)    
  X=check_all_variables(df,ivd,y)
  print(X)

In [97]:
# independent variables

X = []  
if all_variables is None:
  X=get_independent_variables(df, target)  
else: 
  ivd=get_all_variables_csv(all_variables)    
  X=check_all_variables(df, ivd)


X=check_X(X,df)


# Add independent variables

meta_data['X']=X  


# impute missing values

_=impute_missing_values(df,X, scale)

In [98]:
if analysis == 3:
  classification=False
elif analysis == 2:
  classification=True
elif analysis == 1:
  classification=True

In [99]:
# Force target to be factors
# Only 'int' or 'string' are allowed for asfactor(), got Target (Total orders):real 

if classification:
    df[y] = df[y].asfactor()

In [100]:
def check_y(y,df):
  ok=False
  C = [name for name in df.columns if name == y]
  for key, val in df.types.items():
    if key in C:
      if val in ['real','int','enum']:        
        ok=True         
  return ok   

In [101]:
ok=check_y(y,df)
if not ok:
    print(ok)

In [102]:
print(ok)

True


In [103]:
classification=True
if classification:
    print(df[y].levels())

[]


In [104]:
print(df[y].levels())

[]


In [105]:
allV=get_variables_types(df)
allV

{'PassengerId': 'int',
 'Survived': 'int',
 'Pclass': 'int',
 'Name': 'string',
 'Sex': 'enum',
 'Age': 'real',
 'SibSp': 'int',
 'Parch': 'int',
 'Ticket': 'int',
 'Fare': 'real',
 'Cabin': 'enum',
 'Embarked': 'enum'}

In [106]:
meta_data['variables']=allV

In [107]:
# Set up AutoML

aml = H2OAutoML(max_runtime_secs=run_time,project_name = name)

In [108]:
model_start_time = time.time()

In [109]:
aml.train(x=X,y=y,training_frame=df)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [110]:
meta_data['model_execution_time'] = time.time() - model_start_time

In [111]:
# get leaderboard
aml_leaderboard_df=aml.leaderboard.as_data_frame()

In [112]:
aml_leaderboard_df

,model_id,mean_residual_deviance,rmse,mse,mae,rmsle
0,StackedEnsemble_AllModels_AutoML_20181214_171415,145.986803,12.082500,145.986803,9.653954,0.537688
1,StackedEnsemble_BestOfFamily_AutoML_20181214_171415,148.794055,12.198117,148.794055,9.715667,0.547431
2,GBM_1_AutoML_20181214_171415,151.040373,12.289848,151.040373,9.609440,0.530786
3,DeepLearning_grid_1_AutoML_20181214_171415_model_4,152.873410,12.364199,152.873410,9.738334,0.539381
4,GBM_grid_1_AutoML_20181214_171415_model_6,154.960324,12.448306,154.960324,9.819966,0.552667
5,DeepLearning_grid_1_AutoML_20181214_171415_model_1,155.347497,12.463848,155.347497,9.919074,NaN
6,GBM_grid_1_AutoML_20181214_171415_model_10,155.900627,12.486017,155.900627,9.823457,0.556251
7,GBM_3_AutoML_20181214_171415,157.146871,12.535823,157.146871,9.856006,0.547667
8,GBM_2_AutoML_20181214_171415,158.074865,12.572783,158.074865,9.956603,0.555240
9,DeepLearning_1_AutoML_20181214_171415,158.138707,12.575321,158.138707,10.006391,0.590403


In [113]:
# STart best model as first model

model_set=aml_leaderboard_df['model_id']
mod_best=h2o.get_model(model_set[0])

In [114]:
mod_best._id

'StackedEnsemble_AllModels_AutoML_20181214_171415'

In [115]:
# Get stacked ensemble  
se=get_stacked_ensemble(model_set)

In [116]:
print(se)

StackedEnsemble_BestOfFamily_AutoML_20181214_171415


In [117]:
if se is not None:
  mod_best=h2o.get_model(se)

In [118]:
dir(mod_best)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_bc',
 '_bcin',
 '_check_targets',
 '_compute_algo',
 '_estimator_type',
 '_future',
 '_get_metrics',
 '_have_mojo',
 '_have_pojo',
 '_id',
 '_is_xvalidated',
 '_job',
 '_keyify_if_h2oframe',
 '_make_model',
 '_metrics_class',
 '_model_json',
 '_parms',
 '_plot',
 '_requires_training_frame',
 '_resolve_model',
 '_verify_training_frame_params',
 '_xval_keys',
 'actual_params',
 'aic',
 'algo',
 'auc',
 'base_models',
 'biases',
 'catoffsets',
 'coef',
 'coef_norm',
 'convert_H2OXGBoostParams_2_XGBoostParams',
 'cross_validation_fold_assignment',
 'cross_validation_holdout_predictions',
 'cross_validation_m

In [119]:
mod_best._id

'StackedEnsemble_BestOfFamily_AutoML_20181214_171415'

In [120]:
mod_best._get_metrics

<function h2o.model.model_base.ModelBase._get_metrics(o, train, valid, xval)>

In [121]:
type(mod_best)

h2o.estimators.stackedensemble.H2OStackedEnsembleEstimator

In [122]:
mods=mod_best.coef_norm
print(mods)

Model Details
H2OStackedEnsembleEstimator :  Stacked Ensemble
Model Key:  StackedEnsemble_BestOfFamily_AutoML_20181214_171415
No model summary for this model


ModelMetricsRegressionGLM: stackedensemble
** Reported on train data. **

MSE: 69.31267842927711
RMSE: 8.325423618608072
MAE: 6.716328796107707
RMSLE: 0.46164476048299313
R^2: 0.6676679494119901
Mean Residual Deviance: 69.31267842927711
Null degrees of freedom: 637
Residual degrees of freedom: 633
Null deviance: 133064.17108923398
Residual deviance: 44221.488837878795
AIC: 4526.81012989359

ModelMetricsRegressionGLM: stackedensemble
** Reported on validation data. **

MSE: 151.85910492646215
RMSE: 12.323112631411844
MAE: 9.699615426047078
RMSLE: 0.698266700146051
R^2: 0.327299879881343
Mean Residual Deviance: 151.85910492646215
Null degrees of freedom: 75
Residual degrees of freedom: 71
Null deviance: 17420.553624427812
Residual deviance: 11541.291974411124
AIC: 609.4230964241029

ModelMetricsRegressionGLM: stackedensemble
** Re

In [123]:
'''
mod_best.aic()
 mod_best.algo
 mod_best.coef()
 mod_best.coef_norm()
 mod_best.confusion_matrix()
 mod_best.cross_validation_fold_assignment()
 mod_best.cross_validation_holdout_predictions()
 mod_best.cross_validation_metrics_summary()
 mod_best.cross_validation_models()
 mod_best.cross_validation_predictions()
 mod_best.deepfeatures()
 mod_best.default_params()
 mod_best.download_mojo()
 mod_best.download_pojo()
 mod_best.fit()
 mod_best.full_parameters()
 mod_best.get_params()
 mod_best.get_xval_models()
 mod_best.gini()
 mod_best.have_mojo()
 mod_best.have_pojo()
 mod_best.hit_ratio_table()
 mod_best.is_cross_validated()
 mod_best.join()
 mod_best.keep_levelone_frame()
 mod_best.levelone_frame_id()
 mod_best.logloss()
 mod_best.mae()
 mod_best.mean_per_class_error()
 mod_best.mean_residual_deviance()
 mod_best.metalearner()
 mod_best.metalearner_algorithm()
 mod_best.metalearner_fold_assignment()
 mod_best.metalearner_fold_column()
 mod_best.metalearner_nfolds()
 mod_best.metalearner_params()
 mod_best.mixin()
 mod_best.model_id()
 mod_best.model_performance()
 mod_best.mse()
 mod_best.normmul()
 mod_best.normsub()
 mod_best.null_degrees_of_freedom()
 mod_best.null_deviance()
 mod_best.params()
 mod_best.parms()
 mod_best.partial_plot()
 mod_best.plot()
 mod_best.pprint_coef()
 mod_best.predict()
 mod_best.predict_leaf_node_assignment()
 mod_best.r2()
 mod_best.residual_degrees_of_freedom()
 mod_best.residual_deviance()
 mod_best.respmul()
 mod_best.response_column()
 mod_best.respsub()
 mod_best.rmse()
 mod_best.rmsle()
 mod_best.rotation()
 mod_best.save_model_details()
 mod_best.save_mojo()
 mod_best.score_history()
 mod_best.scoring_history()
 mod_best.seed()
 mod_best.set_params()
 mod_best.show()
 mod_best.start()
 mod_best.std_coef_plot()
 mod_best.summary()
 mod_best.train()
 mod_best.training_frame()
 mod_best.type()
 mod_best.validation_frame()
 mod_best.varimp()
 mod_best.varimp_plot()
 mod_best.weights()
 mod_best.xval_keys()
 mod_best.xvalsmod_best()
 '''

'\nmod_best.aic()\n mod_best.algo\n mod_best.coef()\n mod_best.coef_norm()\n mod_best.confusion_matrix()\n mod_best.cross_validation_fold_assignment()\n mod_best.cross_validation_holdout_predictions()\n mod_best.cross_validation_metrics_summary()\n mod_best.cross_validation_models()\n mod_best.cross_validation_predictions()\n mod_best.deepfeatures()\n mod_best.default_params()\n mod_best.download_mojo()\n mod_best.download_pojo()\n mod_best.fit()\n mod_best.full_parameters()\n mod_best.get_params()\n mod_best.get_xval_models()\n mod_best.gini()\n mod_best.have_mojo()\n mod_best.have_pojo()\n mod_best.hit_ratio_table()\n mod_best.is_cross_validated()\n mod_best.join()\n mod_best.keep_levelone_frame()\n mod_best.levelone_frame_id()\n mod_best.logloss()\n mod_best.mae()\n mod_best.mean_per_class_error()\n mod_best.mean_residual_deviance()\n mod_best.metalearner()\n mod_best.metalearner_algorithm()\n mod_best.metalearner_fold_assignment()\n mod_best.metalearner_fold_column()\n mod_best.met

In [124]:
bm=stackedensemble_df(aml_leaderboard_df)

In [125]:
bm

['GBM_1_AutoML_20181214_171415',
 'GLM_grid_1_AutoML_20181214_171415_model_1',
 'DRF_1_AutoML_20181214_171415',
 'XRT_1_AutoML_20181214_171415',
 'DeepLearning_grid_1_AutoML_20181214_171415_model_4']

In [126]:
aml_leaderboard_df

,model_id,mean_residual_deviance,rmse,mse,mae,rmsle
0,StackedEnsemble_AllModels_AutoML_20181214_171415,145.986803,12.082500,145.986803,9.653954,0.537688
1,StackedEnsemble_BestOfFamily_AutoML_20181214_171415,148.794055,12.198117,148.794055,9.715667,0.547431
2,GBM_1_AutoML_20181214_171415,151.040373,12.289848,151.040373,9.609440,0.530786
3,DeepLearning_grid_1_AutoML_20181214_171415_model_4,152.873410,12.364199,152.873410,9.738334,0.539381
4,GBM_grid_1_AutoML_20181214_171415_model_6,154.960324,12.448306,154.960324,9.819966,0.552667
5,DeepLearning_grid_1_AutoML_20181214_171415_model_1,155.347497,12.463848,155.347497,9.919074,NaN
6,GBM_grid_1_AutoML_20181214_171415_model_10,155.900627,12.486017,155.900627,9.823457,0.556251
7,GBM_3_AutoML_20181214_171415,157.146871,12.535823,157.146871,9.856006,0.547667
8,GBM_2_AutoML_20181214_171415,158.074865,12.572783,158.074865,9.956603,0.555240
9,DeepLearning_1_AutoML_20181214_171415,158.138707,12.575321,158.138707,10.006391,0.590403


In [127]:
#  Get best_models and coef_norm()
best_models={}
best_models=stackedensemble(mod_best)
bm=[]
if best_models is not None: 
  if 'Intercept' in best_models.keys():
    del best_models['Intercept']
  bm=list(best_models.keys())
else:
  best_models={}
  bm=stackedensemble_df(aml_leaderboard_df)   
  for b in bm:   
    best_models[b]=None

if mod_best.model_id not in bm:
    bm.append(mod_best.model_id)

In [128]:
bm

['GBM_1_AutoML_20181214_171415',
 'DeepLearning_grid_1_AutoML_20181214_171415_model_4',
 'DRF_1_AutoML_20181214_171415',
 'XRT_1_AutoML_20181214_171415',
 'GLM_grid_1_AutoML_20181214_171415_model_1',
 'StackedEnsemble_BestOfFamily_AutoML_20181214_171415']

In [129]:
# Best of Family leaderboard

aml_leaderboard_df=aml_leaderboard_df.loc[aml_leaderboard_df['model_id'].isin(bm)]

In [130]:
aml_leaderboard_df

,model_id,mean_residual_deviance,rmse,mse,mae,rmsle
1,StackedEnsemble_BestOfFamily_AutoML_20181214_171415,148.794055,12.198117,148.794055,9.715667,0.547431
2,GBM_1_AutoML_20181214_171415,151.040373,12.289848,151.040373,9.609440,0.530786
3,DeepLearning_grid_1_AutoML_20181214_171415_model_4,152.873410,12.364199,152.873410,9.738334,0.539381
17,DRF_1_AutoML_20181214_171415,166.018265,12.884808,166.018265,10.236247,0.560546
20,XRT_1_AutoML_20181214_171415,170.633884,13.062691,170.633884,10.257919,0.589472
37,GLM_grid_1_AutoML_20181214_171415_model_1,209.296416,14.467080,209.296416,11.279221,0.688350


In [131]:
# save leaderboard
leaderboard_stats=run_id+'_leaderboard.csv'
aml_leaderboard_df.to_csv(leaderboard_stats)

In [132]:
top=aml_leaderboard_df.iloc[0]['model_id']
print(top)

StackedEnsemble_BestOfFamily_AutoML_20181214_171415


In [133]:
mod_best=h2o.get_model(top)
print(mod_best._id)
print(mod_best.algo)

StackedEnsemble_BestOfFamily_AutoML_20181214_171415
stackedensemble


In [134]:
meta_data['mod_best']=mod_best._id
meta_data['mod_best_algo']=mod_best.algo

In [135]:
meta_data['models']=bm

In [136]:
models_path=os.path.join(run_dir,'models')
for mod in bm:
  try:   
    m=h2o.get_model(mod) 
    h2o.save_model(m, path = models_path)
  except:    
    pass    

In [137]:
print(models_path)

C:\Users\prabh\WXf3Ticdf\models


In [138]:
# GBM
 
mod,mod_id=get_model_by_algo("GBM",best_models)
if mod is not None:
    try:     
        sh_df=mod.scoring_history()
        sh_df.to_csv(run_id+'_gbm_scoring_history.csv') 
    except:
        pass   
    try:     
        stats_gbm={}
        stats_gbm=gbm_stats(mod)
        n=run_id+'_gbm_stats.json'
        dict_to_json(stats_gbm,n)
        print(stats_gbm)
    except:
        pass        

{'algo': 'gbm', 'model_id': 'GBM_1_AutoML_20181214_171415', 'varimp': [('Cabin', 176905.90625, 1.0, 0.34828746381739056), ('PassengerId', 70705.453125, 0.3996783070943964, 0.1392029439207355), ('Pclass', 58239.265625, 0.3292104082873152, 0.11465985816467665), ('SibSp', 48826.41796875, 0.2760021923730995, 0.09612810358966635), ('Fare', 48579.80078125, 0.2746081338437506, 0.09564257048006644), ('Parch', 40474.29296875, 0.2287899473042608, 0.07968467049351542), ('Survived', 19565.40625, 0.1105978125023737, 0.038519831620203025), ('Ticket', 17843.427734375, 0.10086394576989992, 0.035129647862813265), ('Sex', 14574.671875, 0.08238657591456193, 0.028694211577881688), ('Embarked', 12216.0888671875, 0.06905416063341582, 0.02405069847305109)]}


In [139]:
if mod is not None:
    try:    
        sh_df=mod.scoring_history()
        sh_df.to_csv(run_id+'_dl_scoring_history.csv') 
    except:
        pass 
    try:
        stats_dl={}
        stats_dl=dl_stats(mod)
        n=run_id+'_dl_stats.json'
        dict_to_json(stats_dl,n)
        print(stats_dl)
    except:
        pass    
    try:
        cf=mod.confusion_matrix()    
        cf_df.to_csv(run_id+'_dl_confusion_matrix.csv')
    except:
        pass       

{'algo': 'gbm', 'model_id': 'GBM_1_AutoML_20181214_171415', 'varimp': [('Cabin', 176905.90625, 1.0, 0.34828746381739056), ('PassengerId', 70705.453125, 0.3996783070943964, 0.1392029439207355), ('Pclass', 58239.265625, 0.3292104082873152, 0.11465985816467665), ('SibSp', 48826.41796875, 0.2760021923730995, 0.09612810358966635), ('Fare', 48579.80078125, 0.2746081338437506, 0.09564257048006644), ('Parch', 40474.29296875, 0.2287899473042608, 0.07968467049351542), ('Survived', 19565.40625, 0.1105978125023737, 0.038519831620203025), ('Ticket', 17843.427734375, 0.10086394576989992, 0.035129647862813265), ('Sex', 14574.671875, 0.08238657591456193, 0.028694211577881688), ('Embarked', 12216.0888671875, 0.06905416063341582, 0.02405069847305109)]}


In [140]:
# DRF

mod,mod_id=get_model_by_algo("DRF",best_models)
if mod is not None:
    try:     
         sh_df=mod.scoring_history()
         sh_df.to_csv(run_id+'_drf_scoring_history.csv') 
    except:
         pass  
    try: 
         stats_drf={}
         stats_drf=drf_stats(mod)
         n=run_id+'_drf_stats.json'
         dict_to_json(stats_drf,n)
         print(stats_drf)
    except:
         pass     

In [141]:
# XRT

mod,mod_id=get_model_by_algo("XRT",best_models)
if mod is not None:
    try:     
         sh_df=mod.scoring_history()
         sh_df.to_csv(run_id+'_xrt_scoring_history.csv')
    except:
         pass     
    try:        
         stats_xrt={}
         stats_xrt=xrt_stats(mod)
         n=run_id+'_xrt_stats.json'
         dict_to_json(stats_xrt,n)
         print(stats_xrt)
    except:
         pass     

In [142]:
# GLM

mod,mod_id=get_model_by_algo("GLM",best_models)
if mod is not None:
    try:     
         stats_glm={}
         stats_glm=glm_stats(mod)
         n=run_id+'_glm_stats.json'
         dict_to_json(stats_glm,n)
         print(stats_glm)
    except:
         pass     

{'algo': 'glm', 'model_id': 'GLM_grid_1_AutoML_20181214_171415_model_1', 'coef': {'Intercept': 29.931577388309464, 'Cabin.A10': 1.7889885595391946e-05, 'Cabin.A14': 0.0, 'Cabin.A16': 5.327157853594956e-05, 'Cabin.A19': 0.0, 'Cabin.A20': 5.621322818209506e-05, 'Cabin.A23': 0.0001475026456284589, 'Cabin.A24': 3.166271310578935e-06, 'Cabin.A26': 7.68087552852573e-05, 'Cabin.A31': 2.9691651567974238e-05, 'Cabin.A32': 0.0, 'Cabin.A34': -7.631909956211258e-05, 'Cabin.A36': 2.6734436332794794e-05, 'Cabin.A5': 0.00012099425832625229, 'Cabin.A6': -5.649198070709256e-06, 'Cabin.A7': 0.0, 'Cabin.B101': 1.4884411775774374e-05, 'Cabin.B102': 0.0, 'Cabin.B18': -4.0979276281272954e-05, 'Cabin.B19': 9.153730191711037e-05, 'Cabin.B20': -3.485005689254223e-05, 'Cabin.B22': 1.7948416865252497e-05, 'Cabin.B28': 0.0001182883455132549, 'Cabin.B3': 3.851613846067888e-05, 'Cabin.B30': 0.00010333176879618606, 'Cabin.B35': -3.4867773694174445e-05, 'Cabin.B37': 8.269214322204722e-05, 'Cabin.B38': 4.4406559732158

In [143]:
# split into training and test for showing how to predict
train, test = df.split_frame([0.8])

In [144]:
predictions_df=predictions_test(mod_best,test,run_id)

stackedensemble prediction progress: |████████████████████████████████████| 100%


In [145]:
predictions_df.head()

predict
47.4265
27.331
42.332
38.7472
27.1912
15.5921
11.7798
28.9408
17.2598
41.6262


In [146]:
predictions_df.describe()

Rows:179
Cols:1




,predict
type,real
mins,8.083272510884955
mean,30.93996337788674
maxs,61.54260600780955
sigma,8.084836963120733
zeros,0
missing,0
0,47.42648392567415
1,27.33097547641846
2,42.331998584554405


In [147]:
# Update and save meta data

meta_data['end_time'] = time.time()
meta_data['execution_time'] = meta_data['end_time'] - meta_data['start_time']
  
n=run_id+'_meta_data.json'
dict_to_json(meta_data,n)

In [148]:
meta_data

{'start_time': 1544825621.785323,
 'target': None,
 'server_path': 'C:\\Users\\prabh',
 'data_path': 'C:/Users/prabh/Desktop/ML/Project ML/Dataset/Titanic_train.csv',
 'test_path': None,
 'max_models': 9,
 'run_time': 100,
 'run_id': 'WXf3Ticdf',
 'scale': False,
 'classification': False,
 'model_path': None,
 'balance': False,
 'balance_threshold': 0.2,
 'project': None,
 'end_time': 1544825788.8741882,
 'execution_time': 167.08886528015137,
 'run_path': 'C:\\Users\\prabh\\WXf3Ticdf',
 'nthreads': 1,
 'min_mem_size': 4,
 'analysis': 0,
 'X': ['PassengerId',
  'Survived',
  'Pclass',
  'SibSp',
  'Parch',
  'Ticket',
  'Sex',
  'Cabin',
  'Embarked',
  'Name',
  'Fare'],
 'variables': {'PassengerId': 'int',
  'Survived': 'int',
  'Pclass': 'int',
  'Name': 'string',
  'Sex': 'enum',
  'Age': 'real',
  'SibSp': 'int',
  'Parch': 'int',
  'Ticket': 'int',
  'Fare': 'real',
  'Cabin': 'enum',
  'Embarked': 'enum'},
 'model_execution_time': 89.31195330619812,
 'mod_best': 'StackedEnsemble_

In [149]:
# Save logs
h2o.download_all_logs(dirname=logs_path, filename=logfile)

Writing H2O logs to C:\Users\prabh\WXf3Ticdf\logs\WXf3Ticdf_autoh2o_log.zip


'C:\\Users\\prabh\\WXf3Ticdf\\logs\\WXf3Ticdf_autoh2o_log.zip'

In [150]:
# Clean up
os.chdir(server_path)

In [151]:
def predictions_test(mod,test,run_id):
    mod_perf=mod_best.model_performance(test)          
    stats_test={}
    stats_test=model_performance_stats(mod_perf)
    print(stats_test)   
    n=run_id+'_test_stats.json'
    dict_to_json(stats_test,n) 
    try:
      cf=mod_perf.confusion_matrix()
      print(cf)   
#      cf=mod_perf.confusion_matrix(metrics=["f1","f2","f0point5","accuracy","precision","recall","specificity","absolute_mcc","min_per_class_accuracy","mean_per_class_accuracy"])
      cf_df=cf.table.as_data_frame()
      print(cf_df)    
      cf_df.to_csv(run_id+'_test_confusion_matrix.csv')
    except:
      pass
    predictions = mod_best.predict(test)    
    predictions_df=test.cbind(predictions).as_data_frame() 
    predictions_df.to_csv(run_id+'_predictions.csv')
    return predictions

In [152]:
h2o.cluster().shutdown()

H2O session _sid_bdc4 closed.
